In [1]:
import pandas as pd
import geopandas as gpd
import cartoframes
import carto
import numpy as np
import requests
import json
import os
import codecs

In [2]:
from carto.datasets import DatasetManager

In [3]:
cc = cartoframes.CartoContext()

In [4]:
api_key = cc.creds.key()

In [5]:
BASE_URL = "https://{organization}.carto.com/user/{user}/". \
    format(organization='team',
           user='michellemho-carto')

In [6]:
from carto.auth import APIKeyAuthClient
auth_client = APIKeyAuthClient(api_key=api_key, base_url=BASE_URL)

In [7]:
df = cc.read('carto_homes',decode_geom=True)

In [8]:
# Cutoffs every 10 minutes from 10 minutes to 90 minutes
query = '''
http://localhost:8080/otp/routers/default/isochrone?
batch=True&
mode=WALK,TRANSIT&
date=04/13/2018&
time=8:00am&
maxWalkDistance=1600&
maxTransfers=4&
cutoffSec=600&
cutoffSec=1200&
cutoffSec=1800&
cutoffSec=2400&
cutoffSec=3000&
cutoffSec=3600&
cutoffSec=4200&
cutoffSec=4800&
cutoffSec=5400&
fromPlace=
'''
# sample origin point
origin = '40.71708,-73.95172'

# print(query.strip() + origin)
query_constructed = query.replace('\n','') + origin

In [9]:
query_constructed

'http://localhost:8080/otp/routers/default/isochrone?batch=True&mode=WALK,TRANSIT&date=04/13/2018&time=8:00am&maxWalkDistance=1600&maxTransfers=4&cutoffSec=600&cutoffSec=1200&cutoffSec=1800&cutoffSec=2400&cutoffSec=3000&cutoffSec=3600&cutoffSec=4200&cutoffSec=4800&cutoffSec=5400&fromPlace=40.71708,-73.95172'

In [31]:
df.head()

,address_line_1,state,the_geom,zip_code,geometry
cartodb_id,,,,,
1,796 Madison St,NY,0101000020E61000002D793C2D3F7B52C0841266DAFE57...,11221,POINT (-73.925731 40.687465)
2,157 Green St,NY,0101000020E61000007D5EF1D4237D52C0220038F6EC5D...,11222,POINT (-73.95531200000001 40.733794)
3,1096 President St Apt 44,NY,0101000020E61000000B9A9658197D52C0AFEB17EC8655...,11225,POINT (-73.954672 40.66818)
4,4720 Center Blvd,NY,0101000020E61000009241EE224C7D52C0FE65F7E4615F...,11109,POINT (-73.95777200000001 40.745175)
6,301 W 105th Street,NY,0101000020E61000000CE544BB0A7E52C0587380608E66...,10025,POINT (-73.96940499999999 40.80122)


In [11]:
banned_stops = []
stop_ids = ["1:L01N",
"1:L01S",
"1:L02N",
"1:L02S",
"1:L03N",
"1:L03S",
"1:L05N",
"1:L05S",
"1:L06N",
"1:L06S"]
for i in stop_ids:
    banned_stops.append("MTA NYCT_"+i)

In [12]:
for i in stop_ids:
    print(i+',')

1:L01N,
1:L01S,
1:L02N,
1:L02S,
1:L03N,
1:L03S,
1:L05N,
1:L05S,
1:L06N,
1:L06S,


In [13]:
for i in banned_stops:
    print(i+',')

MTA NYCT_1:L01N,
MTA NYCT_1:L01S,
MTA NYCT_1:L02N,
MTA NYCT_1:L02S,
MTA NYCT_1:L03N,
MTA NYCT_1:L03S,
MTA NYCT_1:L05N,
MTA NYCT_1:L05S,
MTA NYCT_1:L06N,
MTA NYCT_1:L06S,


In [29]:
# Cutoffs every 10 minutes to 90 minutes WITH BANNED STOPS HARD

query_no_L = '''
http://localhost:8080/otp/routers/default/isochrone?
mode=WALK,TRANSIT&
date=04/13/2018&
time=8:00am&
bannedStopsHard=1:L01N,
1:L01S,
1:L02N,
1:L02S,
1:L03N,
1:L03S,
1:L05N,
1:L05S,
1:L06N,
1:L06S&
maxWalkDistance=1600&
maxTransfers=4&
cutoffSec=600&
cutoffSec=1200&
cutoffSec=1800&
cutoffSec=2400&
cutoffSec=3000&
cutoffSec=3600&
cutoffSec=4200&
cutoffSec=4800&
cutoffSec=5400&
routerId=default&
fromPlace=
'''.format(banned_stops=banned_stops)
origin = '40.71708,-73.95172'

# print(query.strip() + origin)
query_constructed_no_L = query_no_L.replace('\n','') + origin

In [30]:
query_constructed_no_L

'http://localhost:8080/otp/routers/default/isochrone?mode=WALK,TRANSIT&date=04/13/2018&time=8:00am&bannedStopsHard=1:L01N,1:L01S,1:L02N,1:L02S,1:L03N,1:L03S,1:L05N,1:L05S,1:L06N,1:L06S&maxWalkDistance=1600&maxTransfers=4&cutoffSec=600&cutoffSec=1200&cutoffSec=1800&cutoffSec=2400&cutoffSec=3000&cutoffSec=3600&cutoffSec=4200&cutoffSec=4800&cutoffSec=5400&routerId=default&fromPlace=40.71708,-73.95172'

In [32]:
# add address and origin to properties
# save all geojson isocrhones in new directory "geojson_folder_no_L" (make this directory first)


for i in df.iterrows():
    long = i[1]['geometry'].x
    lat = i[1]['geometry'].y
    address = i[1]['address_line_1']
    origin = str(lat) +','+str(long)
    query_constructed_no_L = query_no_L.replace('\n','') + origin
    try:
        response = requests.request('GET', query_constructed_no_L)
        data = response.json()
        for d in data['features']:
            d['properties']['origin'] = origin
            d['properties']['address'] = address

        with open('nyc_w_bus/output_{}.geojson'.format(i[0]), 'w') as f:
            json.dump(data, f)
    except:
        print(i[1],i[0],'failed')
        continue
#     LOCAL_FILE_OR_URL = 'output_{}.geojson'.format(i[0])
#     dataset_manager = DatasetManager(auth_client)
#     dataset = dataset_manager.create(LOCAL_FILE_OR_URL)

address_line_1                                        722 League St
state                                                            PA
the_geom          0101000020E61000004BE7C3B304CA52C0AA61BF27D6F7...
zip_code                                                      19147
geometry                               POINT (-75.156537 39.936223)
Name: 20, dtype: object 20 failed
address_line_1                                       84 Prescott St
state                                                            MA
the_geom          0101000020E610000055A4C2D842C751C0C2C3B46FEE2F...
zip_code                                                      02138
geometry                               POINT (-71.113455 42.374464)
Name: 21, dtype: object 21 failed


In [20]:
df.shape

(22, 5)

In [22]:
df.index

Int64Index([1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 22,
            23, 7, 21, 24],
           dtype='int64', name='cartodb_id')

In [33]:
# merge all the geojsons together into one ... use geojson-merge https://github.com/mapbox/geojson-merge
os.system('geojson-merge nyc_w_bus/*.geojson > combined_w_bus.geojson')

0

In [34]:
# Use CARTO python manager to send file to your CARTO account (authenticated before)

LOCAL_FILE_OR_URL = 'combined_w_bus.geojson'
dataset_manager = DatasetManager(auth_client)
dataset = dataset_manager.create(LOCAL_FILE_OR_URL)

/Users/mho/anaconda3/lib/python3.5/site-packages/carto/resources.py:90: FutureWarning: This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment
  warnings.warn('This is part of a non-public CARTO API and may change in the future. Take this into account if you are using this in a production environment', FutureWarning)


In [35]:
# Check out the isochrone layer...

cc.map(layers=cartoframes.Layer('combined_w_bus'))

In [264]:
rect_grid_creation = '''
CREATE TABLE rect_grid_copy
AS (SELECT ST_Transform(ST_SetSRID(CDB_RectangleGrid(ST_Extent(the_geom), .002, .002),4326),3857)  as the_geom_webmercator, ST_SetSRID(CDB_RectangleGrid(ST_Extent(the_geom), .002, .002),4326) as the_geom FROM combined_no_L)
'''

In [36]:
# Add the cartodb_id and make accessible in dashboard
cartodbfytable = '''
SELECT CDB_cartodbfytable('michellemho-carto', 'rect_grid_copy')
'''

In [37]:
df = cc.read('rect_grid_copy')

In [38]:
cc.map(layers=cartoframes.Layer('rect_grid_extended'))

In [39]:
cc.map(layers=cartoframes.QueryLayer('select * from rect_grid_extended where count = 20'))

In [178]:
# This is just for testing
count_overlaps ='''
  SELECT count(*) AS count, p.cartodb_id AS id  
  FROM rect_grid p 
  JOIN combined_1 c 
  ON ST_intersects(c.the_geom, p.the_geom)
  GROUP BY p.cartodb_id
'''

In [38]:
# WARNING: THIS WILL TIMEOUT
# USE BATCH API TO RUN THESE UPDATE STATMENTS https://cartodb.github.io/customer_success/batch/

# cc.query(update_count_overlaps)

In [270]:
# use distinct on c.origin to avoid double counting ids and times...

update_count_overlaps = '''
  ALTER TABLE grid_test ADD COLUMN count INTEGER DEFAULT 0;
UPDATE grid_test set count = p.count
FROM
  (SELECT count(distinct(c.origin)) AS count, p.cartodb_id AS id  
  FROM grid_test p 
  JOIN extended_no_L c 
  ON ST_intersects(c.the_geom, p.the_geom)
  GROUP BY p.cartodb_id) as p
WHERE p.id = grid_test.cartodb_id
'''

In [26]:
update_sum_overlaps_minimums = '''
ALTER TABLE rect_grid_extended ADD COLUMN sum_time INTEGER DEFAULT 0;
UPDATE rect_grid_extended set sum_time = d.sum
FROM (
SELECT sum(min_table.minimum), min_table.id AS id
FROM
  (SELECT min(time) AS minimum, p.cartodb_id AS id, c.address
  FROM rect_grid_extended p 
  JOIN extended_no_L c 
  ON ST_Intersects(c.the_geom,p.the_geom)
  GROUP BY p.cartodb_id, c.address) as min_table
GROUP BY min_table.id) as d
where d.id = rect_grid_extended.cartodb_id;
'''

# Trying just a planned trips

In [28]:
# trip to Andy
plan_query = '''
http://localhost:8080/otp/routers/default/plan?
toPlace=40.8012,-73.9694&
date=04/13/2018&
time=8:00am&
mode=BUS,WALK&
maxWalkDistance=1600&
arriveBy=false&
bannedStops=1:L10N&
fromPlace=
'''
print(plan_query+origin)


http://localhost:8080/otp/routers/default/plan?
toPlace=40.8012,-73.9694&
date=04/13/2018&
time=8:00am&
mode=BUS,WALK&
maxWalkDistance=1600&
arriveBy=false&
bannedStops=1:L10N&
fromPlace=
40.75065443,-73.99370313
